# Logistic regression to predict tip category

Obtain necessary data from the two datafiles:

In [1]:
import csv
infile = open("nyc_data.csv", 'r')    # open the file for reading
infile2 = open("nyc_fare.csv", 'r')

reader = csv.reader(infile)
reader2 = csv.reader(infile2)
reader.next() #remove headers
reader2.next()

taxi_file = []
taxi_file2 = []

for line in reader:     
    taxi_file.append(line)
    
nb = 0
for line2 in reader2:                   # read through the CSV one line at a time
    taxi_file2.append(line2)
    
print len(taxi_file)

846945


In [15]:
temp =[]
data = []

for line in taxi_file:
    data_list = []
    data_list.append(float(line[7]))
    data_list.append(float(line[8]))
    data_list.append(float(line[9]))
    temp.append(data_list)
    
nb = 0
for line2 in taxi_file2:                   # read through the CSV one line at a time
    if line2[4] == 'CRD':
        data.append(temp[nb]+ ([float(line2[5]) + float(line2[6]) + float(line2[7]) + float(line2[9])] + ([line2[8]])))
    nb+=1
    
print len(data)
print data[0]

456019
[1.0, 960.0, 3.28, 14.5, '2.8']


Transform tip value in categories:

In [16]:
import numpy as np
from __future__ import division

y_mean = np.mean([float(d[4]) for d in data])
y_low = y_mean * 2/5
y_medium_low = y_low * 2
y_medium = y_mean * 6/5
y_medium_high = y_mean * 8/5

print y_low
print y_medium_low
print y_medium
print y_medium_high

i=0
for d in data:
    if float(d[4]) < y_low:
        data[i][4] = 'low'
    elif float(d[4]) < y_medium_low:
        data[i][4] = 'medium-low'
    elif float(d[4]) < y_medium:
        data[i][4] = 'medium'
    elif float(d[4]) < y_medium_high:
        data[i][4] = 'medium-high'
    else:
        data[i][4] = 'high'
    i+=1 
    
print data[12][4]

0.996702277756
1.99340455551
2.99010683327
3.98680911102
high


Divide in training and testing set:

In [17]:
import random

train = []
test = []

for d in data:
    rd = random.random()
    if rd < 0.8:
        train.append(d)
    elif rd < 1.0:
        test.append(d)
        
print 'train ',len(train)
print 'test ',len(test)

train  365077
test  90942


Get input and output variables for the training set:

In [18]:
x = [line[:4] for line in train]
y = [line[4] for line in train]

Normalize the training data:

In [19]:
x_mean = np.array(x).mean(axis=0)
x_std = np.array(x).std(axis=0)

i=0
for line in x:
    x[i][0] = (line[0] - x_mean[0])/x_std[0]
    x[i][1] = (line[1] - x_mean[1])/x_std[1]
    x[i][2] = (line[2] - x_mean[2])/x_std[2]
    x[i][3] = (line[3] - x_mean[3])/x_std[3]   
    i += 1

Perform multivariate regression:

In [20]:
from sklearn import datasets, linear_model

regr = linear_model.LogisticRegression()
regr.fit(x, y)

print'Coefficients: ', regr.coef_

Coefficients:  [[ -2.45891799e-02  -2.70538712e-04   4.24727755e-01   2.18049957e+00]
 [  5.38067210e-02   2.47790545e-03  -1.60766813e-01  -6.23901113e-02]
 [ -7.32308121e-03   3.49164752e-03  -3.23209435e-01   2.98620391e-02]
 [ -2.82916653e-03   2.98309003e-03  -1.29312597e-01   3.34242790e-01]
 [  7.61907215e-03  -5.78961058e-03   1.59674701e-01  -3.61795718e+00]]


Prepare the testing set:

In [21]:
x_test = [line[:4] for line in test]
y_test = [line[4] for line in test]

i = 0
for line in x_test:
    x_test[i][0] = (line[0] - x_mean[0])/x_std[0]
    x_test[i][1] = (line[1] - x_mean[1])/x_std[1]
    x_test[i][2] = (line[2] - x_mean[2])/x_std[2]
    x_test[i][3] = (line[3] - x_mean[3])/x_std[3]
    i += 1

Get prediction for the testing set:

In [22]:
prediction = regr.predict(x_test)

Get performance score on testing set:
    

In [23]:
print regr.score(x_test, y_test)

0.579897077258


The $r^2$ score is not that bad, but there is still room for improvement. As the tip depends also on the passenger's experience during the trip, maybe taking the taxi driver's identifier could improve the score? Maybe some drivers are nicer and drive better than other, and are thus more likely to get big tips?

In [14]:
temp =[]
data = []

for line in taxi_file:
    data_list = []
    data_list.append(line[0])
    data_list.append(line[1])
    data_list.append(line[2])
    data_list.append(float(line[7]))
    data_list.append(float(line[8]))
    data_list.append(float(line[9]))
    temp.append(data_list)
    
nb = 0
for line2 in taxi_file2:                   # read through the CSV one line at a time
    if line2[4] == 'CRD':
        data.append(temp[nb]+ ([float(line2[5]) + float(line2[6]) + float(line2[7]) + float(line2[9])] + ([line2[8]])))
    nb+=1
    
print len(data)
print data[0]

456019
['517C6B330DBB3F055D007B07512628B3', '2C19FBEE1A6E05612EFE4C958C14BC7F', 'VTS', 1.0, 960.0, 3.28, 14.5, '2.8']


Now that we have a new dataset, we can perform exactly the same step than before: divide in training/testing set, fit a logisitic regression, and look at $r^2$ score

In [25]:
import numpy as np
from __future__ import division
    
train = []
test = []

for d in data:
    rd = random.random()
    if rd < 0.8:
        train.append(d)
    elif rd < 1.0:
        test.append(d)
        
print 'train ',len(train)
print 'test ',len(test)

x = [line[:4] for line in train]
y = [line[4] for line in train]

x_mean = np.array(x).mean(axis=0)
x_std = np.array(x).std(axis=0)

i=0
for line in x:
    x[i][0] = (line[0] - x_mean[0])/x_std[0]
    x[i][1] = (line[1] - x_mean[1])/x_std[1]
    x[i][2] = (line[2] - x_mean[2])/x_std[2]
    x[i][3] = (line[3] - x_mean[3])/x_std[3]   
    i += 1
    
regr = linear_model.LogisticRegression()
regr.fit(x, y)

print'Coefficients: ', regr.coef_

x_test = [line[:4] for line in test]
y_test = [line[4] for line in test]

i = 0
for line in x_test:
    x_test[i][0] = (line[0] - x_mean[0])/x_std[0]
    x_test[i][1] = (line[1] - x_mean[1])/x_std[1]
    x_test[i][2] = (line[2] - x_mean[2])/x_std[2]
    x_test[i][3] = (line[3] - x_mean[3])/x_std[3]
    i += 1
    
print regr.score(x_test, y_test)

train  364792
test  91227
Coefficients:  [[ -2.50341253e-02  -5.42387582e-04   3.83419640e-01   2.24378817e+00]
 [  5.46517942e-02   4.03958395e-03  -1.27034808e-01  -1.13012099e-01]
 [ -5.37809397e-03   6.41685153e-03  -3.20323444e-01   2.81177428e-02]
 [ -7.06154974e-03   4.86454301e-03  -1.32456649e-01   3.36474543e-01]
 [  7.95958892e-03  -6.70435627e+00  -8.18809378e-02  -3.02715071e+00]]
0.577844278558


This has definitly not helped a lot. Maybe the logisitic regression approach is not the best: let's try with a decision tree classifier.

For this, we will wome back to the first dataset, using the number of passengers, the trip duration, the trip distance, the amount of the trip, based only on trips paid with credit cards.

In [28]:
temp =[]
data = []

for line in taxi_file:
    data_list = []
    data_list.append(float(line[7]))
    data_list.append(float(line[8]))
    data_list.append(float(line[9]))
    temp.append(data_list)
    
nb = 0
for line2 in taxi_file2:                   # read through the CSV one line at a time
    if line2[4] == 'CRD':
        data.append(temp[nb]+ ([float(line2[5]) + float(line2[6]) + float(line2[7]) + float(line2[9])] + ([line2[8]])))
    nb+=1
    
y_mean = np.mean([float(d[4]) for d in data])
y_low = y_mean * 2/5
y_medium_low = y_low * 2
y_medium = y_mean * 6/5
y_medium_high = y_mean * 8/5

print y_low
print y_medium_low
print y_medium
print y_medium_high

i=0
for d in data:
    if float(d[4]) < y_low:
        data[i][4] = 'low'
    elif float(d[4]) < y_medium_low:
        data[i][4] = 'medium-low'
    elif float(d[4]) < y_medium:
        data[i][4] = 'medium'
    elif float(d[4]) < y_medium_high:
        data[i][4] = 'medium-high'
    else:
        data[i][4] = 'high'
    i+=1 
    
import random

train = []
test = []

for d in data:
    rd = random.random()
    if rd < 0.8:
        train.append(d)
    elif rd < 1.0:
        test.append(d)
        
print 'train ',len(train)
print 'test ',len(test)

x = [line[:4] for line in train]
y = [line[4] for line in train]

0.996702277756
1.99340455551
2.99010683327
3.98680911102
train  364270
test  91749


Now let's fit a decision tree to the training set:

In [30]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Now let's get the score of the decision tree for the testing set:

In [31]:
x_test = [line[:4] for line in test]
y_test = [line[4] for line in test]

print clf.score(x_test, y_test)

0.594971062355


In [34]:
from sklearn import ensemble

forest = ensemble.RandomForestClassifier()
forest.fit(x,y)
clf.score(x_test, y_test)

0.59497106235490305